In [1]:
import Parser, Deriver, Interpreter, Analyser, SelfOrg
import numpy as np
import math

In [2]:
modelName = 'lentest'

In [3]:
# Have to generate manually as we want points to be same for render and stats.
# We also want to be able to access some inner stuff for plotting

In [4]:
Specification   = Parser.parse('Specifications/'+modelName+'.txt', {})

In [5]:
scale = 1

In [6]:
Specification

{'age_limit': 1000,
 'gen_limit': 1000,
 'self_org': False,
 'defaults': {},
 'sections': {'1': {'constructer': Cell.Sig2,
   'slash': 9.0,
   'slashr': 0.0,
   'plus': 120.0,
   'plusr': 40.0,
   'decay': 0.7,
   'd': 0.5,
   'h': 30.0,
   'w': 0.15,
   'rules': [['GEOM', [1.0], '0>1']],
   'id': '1',
   'width': 10.0}},
 'axiom': <Cell.Sig2 at 0x7f92e32cab70>,
 'depth': 10,
 'thickness': 10.0,
 'num_voxels': 50,
 'shadow_width': 4,
 'shadow_height': 10,
 'dec_close': 0.98,
 'dec_far': 1.0}

In [7]:
points = []

if Specification['self_org'] == False:

    # DERIVE
    n               = Specification['depth']
    axiom           = Specification['axiom']
    tree            = [axiom]
    tree            = Deriver.derive(tree,n)

    # INTERPRET
    points, scale   = Interpreter.interpret(tree, True, return_scale=True)

else:

    # DERIVE AND INTERPRET TOGETHER
    n               = Specification['depth']
    axiom           = Specification['axiom']
    tree            = [axiom]
    num_voxels      = Specification['num_voxels']
    shadow_height   = Specification['shadow_height']
    shadow_width    = Specification['shadow_width']
    decrement_close = Specification['dec_close']
    decrement_far   = Specification['dec_far']

    for i in range(0,n):
        tree = Deriver.derive(tree,1)
        points, scale = Interpreter.interpret(tree, True, return_scale = True)
        voxels          = SelfOrg.calc_light(points,
                                             num_voxels,
                                             shadow_height,
                                             shadow_width,
                                             decrement_close,
                                             decrement_far)
        SelfOrg.update_cells(tree, voxels, scale)
        print("Interpreted for the "+str(i+1)+"th time.")

In [8]:
# So we can view what we visualised
print("writing to file...")
with open("data.dat", mode="w") as f:
    f.write(str(len(points))+"\n")
    for p in points:
        x,y,z,r,a,b,c,d,l = p
        x /= scale
        y /= scale
        y -= 0.5
        z /= scale
        r /= scale
        line = [x,y,z,r,l]
        for feature in line:
            f.write(str(feature)+"\t")
        f.write("\n")
print("done writing.")

writing to file...
done writing.


In [9]:
points[0]

(0, 0, 0, 10.0, 0.0, 270.0, 0.0, 9.527234285809307, 0.0)

In [10]:
points[1]

(3.5721407054634913e-32,
 9.527234285809307,
 -5.833748486421644e-16,
 7.0,
 0.0,
 270.0,
 0.0,
 9.527234285809307,
 0.0)

In [11]:
points[2]
full_points = points
full_points = np.array(points)

In [12]:
points = points[2:]

In [13]:
metrics         = Analyser.analyse(points)

ValueError: math domain error

In [ ]:
vertices        = np.array(points)

In [ ]:
for (k,v) in metrics.items():
    print(k, ':', v)

In [ ]:
math.degrees(metrics['parentAngle_mean'])

In [ ]:
vertices[:,5].mean()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Fed up of accessing via numerical values
# Format of points is:
# 0--1--2--3--4--5--6--7--8
# x  y  z  r  vA hA pA l--leaf
keys = {
    'x': 0,
    'y': 1,
    'z': 2,
    'radius': 3,
    'vertical orientation': 4,
    'horizontal orientation': 5,
    'parent angle': 6,
    'length': 7,
    'leaf': 8
}

In [ ]:
lens = vertices[:,keys['length']][::-2]
inds = np.asarray(range(0,10))
len(inds)

In [ ]:
plt.scatter(inds, lens, color='blue', edgecolor='black')

In [ ]:
plt.hist(vertices[:,keys['length']], color='blue', edgecolor='black', bins=36, log=False)

In [ ]:
max_height = np.max(vertices[:,keys['y']])

In [ ]:
lower_third  = full_points[full_points[:,keys['y']] < max_height/3]
middle_third = full_points[full_points[:,keys['y']] < 2*max_height/3]
middle_third = middle_third[middle_third[:,keys['y']] > max_height/3]
upper_third = full_points[full_points[:,keys['y']] > 2*max_height/3]

In [ ]:
points[0]

In [ ]:
plt.hist(lower_third[:,keys['y']], color='blue', edgecolor='black', bins=36)

In [ ]:
plt.hist(middle_third[:,keys['x']], color='blue', edgecolor='black', bins=36)

In [ ]:
plt.hist(upper_third[:,keys['x']], color='blue', edgecolor='black', bins=36)

In [ ]:
# Problem with these graphs is that they weight all nodes equally, regardless of size and mass
# Create metric that calculates biomass for each thing, then adds that many of itself to distribution.

weighted_ps = []

In [ ]:
vertices.shape[0]/2

In [ ]:
ps = zip(points[::2], points[1::2])

In [ ]:
points[5]

In [ ]:
weights = np.array([Analyser.volumeOfConic(r1,r2,l) for ((x,y,z,r1,va,ha,pa,l,leaf),(x2,y2,z2,r2,va2,ha2,pa2,l2,leaf)) in ps])

In [ ]:
weights.sum()

In [ ]:
log_weights = [(Analyser.volumeOfConic(r1,r2,l)) for ((x,y,z,r1,va,ha,pa,l,leaf),(x2,y2,z2,r2,va2,ha2,pa2,l2,leaf)) in ps]

In [ ]:
plt.hist(log_weights, color='black', edgecolor='black', bins=36)

In [ ]:
n = weights.shape[0]
n

In [ ]:
import math

In [ ]:
points[2]

In [ ]:
for i in range(n):
    for j in range(math.floor(math.log(weights[i]))):
        weighted_ps.append(points[2*i])

In [ ]:
weighted_ps = np.array(weighted_ps)
weighted_ps.shape
weighted_ps[3]

In [ ]:
plt.hist(weighted_ps[:, keys['parent angle']], bins=36)

In [ ]:
degrees = weighted_ps[:,keys['parent angle']]
radians = np.deg2rad(degrees)

bin_size = 5
a , b=np.histogram(degrees, bins=np.arange(0, 360+bin_size, bin_size))
centers = np.deg2rad(np.ediff1d(b)//2 + b[:-1])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='polar')
ax.bar(centers, a, width=np.deg2rad(bin_size), bottom=0.0, color='.8', edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
plt.show()
plt.savefig("metrics_vo.png")

In [ ]:
leaves = vertices[vertices[:, 8] == 1]

In [ ]:
vertices.shape

In [ ]:
leaves.shape

In [ ]:
plt.gca().set_aspect('equal', adjustable='box')
plt.scatter(upper_third[:,2], upper_third[:,1], color='green')
plt.scatter(middle_third[:,2], middle_third[:,1], color='blue')
plt.scatter(lower_third[:,2], lower_third[:,1], color='red')
plt.show()
plt.savefig('z.png')

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

sequence_containing_x_vals = middle_third[:,0]
sequence_containing_y_vals = middle_third[:,2]
sequence_containing_z_vals = middle_third[:,1]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals, color='blue', alpha=0.05)

sequence_containing_x_vals = upper_third[:,0]
sequence_containing_y_vals = upper_third[:,2]
sequence_containing_z_vals = upper_third[:,1]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals, color='green', alpha=0.1)

sequence_containing_x_vals = lower_third[:,0]
sequence_containing_y_vals = lower_third[:,2]
sequence_containing_z_vals = lower_third[:,1]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals, color='red', alpha=1)

pyplot.show()